In [ ]:
!pip install tensorflow keras scikit-learn pandas

In [1]:
from google.colab import files
uploaded = files.upload()

Saving eda_123.csv to eda_123.csv


In [2]:
import pandas as pd

data = pd.read_csv('eda_123.csv')
print(data.head())

   label                                               text
0      1  leftover the movie was visually stunning and l...
1      1  the movie was visually stupefy and left a last...
2      1  the movie was leftover visually stunning and l...
3      1  the movie was visually stunning and left a las...
4      1  last the movie was visually stunning and left ...


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

X = data['text']
y = data['label']

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tokenizer = Tokenizer(num_words=696, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq, maxlen=100, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=100, padding='post', truncating='post')

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras import regularizers

model = Sequential([

    Embedding(input_dim=696, output_dim=128, input_length=15),
    Conv1D(128, 5, activation='relu', kernel_regularizer=regularizers.l2()),
    Conv1D(16, 5, activation='relu', kernel_regularizer=regularizers.l2()),
    Conv1D(16, 5, activation='relu', kernel_regularizer=regularizers.l2()),
    Conv1D(8, 5, activation='relu', kernel_regularizer=regularizers.l2()),

    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.4),
    Dense(1, activation='sigmoid')
])
model.build(input_shape=(None, 200))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'precision', 'recall', 'auc'])

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 200, 128)            │          89,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_4 (Conv1D)                    │ (None, 196, 128)            │          82,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_5 (Conv1D)                    │ (None, 192, 16)             │          10,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_6 (Conv1D)                    │ (None, 188, 16)             │           1,296 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_7 (Conv1D)                    │ (None, 184, 8)              │             648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_1               │ (None, 8)                   │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │             576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 183,977 (718.66 KB)

 Trainable params: 183,977 (718.66 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
history = model.fit(
    X_train_padded,
    y_train,
    epochs=17,
    batch_size=32,
    validation_data=(X_test_padded, y_test)
)

Epoch 1/17
26/26 ━━━━━━━━━━━━━━━━━━━━ 6s 113ms/step - accuracy: 0.5135 - auc: 0.5334 - loss: 2.1512 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.4904 - val_auc: 0.7972 - val_loss: 1.3176 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/17
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - accuracy: 0.5793 - auc: 0.8874 - loss: 1.1765 - precision: 0.8815 - recall: 0.1417 - val_accuracy: 0.8654 - val_auc: 0.9995 - val_loss: 0.8688 - val_precision: 1.0000 - val_recall: 0.7358
Epoch 3/17
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - accuracy: 0.9160 - auc: 0.9987 - loss: 0.7359 - precision: 0.9991 - recall: 0.8359 - val_accuracy: 0.9904 - val_auc: 0.9998 - val_loss: 0.4759 - val_precision: 1.0000 - val_recall: 0.9811
Epoch 4/17
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.9985 - auc: 0.9986 - loss: 0.3851 - precision: 0.9971 - recall: 1.0000 - val_accuracy: 0.9952 - val_auc: 1.0000 - val_loss: 0.1696 - val_precision: 0.9907 - val_recall: 1.0000
Epoch 5/17
26/26 ━━

In [16]:
loss, accuracy ,precision, recall, auc = model.evaluate(X_test_padded, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 1.0000 - auc: 1.0000 - loss: 0.0520 - precision: 1.0000 - recall: 1.0000
Test Accuracy: 1.0000


In [17]:
# I didnt change the layers of this CNN, I just change the input number of unique words